In [1]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama  # Solution locale
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import warnings
warnings.filterwarnings('ignore')

c:\Users\Axel\anaconda3\envs\mini_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Charger vos documents
loader = PyPDFLoader("TP8_cnn.pdf")  # ou TextLoader, DirectoryLoader
documents = loader.load()

# 2. Découper en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # taille des chunks
    chunk_overlap=200,    # chevauchement pour garder le contexte
    length_function=len
)
chunks = text_splitter.split_documents(documents)

print(f"Nombre de chunks créés : {len(chunks)}")

Nombre de chunks créés : 13


In [3]:
# 1. Choisir le modèle d'embedding
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}  # 'cuda' si GPU
)

# 2. Créer la base vectorielle FAISS
vector_store = FAISS.from_documents(chunks, embedding_model)

# 3. Sauvegarder la base
vector_store.save_local("faiss_index")

print("Base FAISS créée et sauvegardée !")

Base FAISS créée et sauvegardée !


In [4]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [6]:
llm = Ollama(
    model="mistral",
    temperature=0.1,  # Pour des réponses plus factuelles
    num_predict=512   # Limite la longueur des réponses
)

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [8]:
question = "Quelle est la motivation du TP8 Deep Learning et Réseau de Neurones Convolutionnel ?"
response = qa_chain.run(question)
print(response)

 The motivation of TP8 (Deep Learning and Convolutional Neural Networks) is to implement a variant of neural networks that are particularly useful for image processing. This variant, known as CNN (Convolutional Neural Network), will be learned, used, and analyzed. However, the focus will be on an Arduino DUE which has less computing power compared to computers.
